In [5]:
import xarray as xr
import glob

path_data = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m/*.nc"

fichiers = sorted(glob.glob(path_data))
print(f"Fichiers trouvés : {len(fichiers)}")

if len(fichiers) > 0:
    print(f"\n🧪 Test du premier fichier : {fichiers[0]}")
    try:
        ds_test = xr.open_dataset(fichiers[0])
        print("✅ Fichier lisible !")
        print(ds_test)
    except Exception as e:
        print(f"❌ Erreur : {e}")

Fichiers trouvés : 912

🧪 Test du premier fichier : C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m\era5_t2m_fr_1950_01.nc
✅ Fichier lisible !
<xarray.Dataset> Size: 8MB
Dimensions:     (valid_time: 744, latitude: 43, longitude: 62)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 6kB 1950-01-01 ... 1950-01-31T23:...
  * latitude    (latitude) float64 344B 51.5 51.25 51.0 ... 41.5 41.25 41.0
  * longitude   (longitude) float64 496B -5.5 -5.25 -5.0 -4.75 ... 9.25 9.5 9.75
    number      int64 8B ...
    expver      (valid_time) <U4 12kB ...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 8MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 202

In [ ]:
# Librairies
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os

# CONFIGURATION
path_data = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m/*.nc"
path_mask = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/src/mask_france_boolean.nc"

print("🚀 Démarrage du script de récupération...")

# TRAITEMENT GLOBAL
try:
    # 0. SÉLECTION DES 25 PREMIÈRES ANNÉES
    print("🔍 Sélection des fichiers...")
    fichiers = sorted(glob.glob(path_data))
    print(f"   → {len(fichiers)} fichiers trouvés au total")

    # Prendre les 300 premiers fichiers (25 ans × 12 mois)
    fichiers_25ans = fichiers[:300]
    print(f"   → Sélection de {len(fichiers_25ans)} fichiers (25 premières années)")

    # Affichage des noms de fichiers (correction de la syntaxe)
    premier_fichier = os.path.basename(fichiers_25ans[0])
    dernier_fichier = os.path.basename(fichiers_25ans[-1])
    print(f"   → Premier : {premier_fichier}")
    print(f"   → Dernier : {dernier_fichier}")

    # 1. CHARGEMENT
    print("⏳ Chargement des données (cela peut prendre 1-2 minutes)...")
    ds = xr.open_mfdataset(
        fichiers_25ans,
        chunks={'valid_time': 200},
        parallel=True,
        engine='netcdf4',
        combine='by_coords'
    )

    print(f"✅ Données chargées : {ds.dims}")
    print(f"   Période : {str(ds.valid_time.min().values)[:10]} à {str(ds.valid_time.max().values)[:10]}")

    # Chargement du masque
    print("⏳ Chargement du masque...")
    mask = xr.open_dataset(path_mask)

    # 2. MASQUAGE
    print("✂️ Application du masque France...")
    ds_france = ds.where(mask['mask'] == 1)

    # --- FICHIER 1 : L'HISTORIQUE (CSV) ---
    print("📊 Calcul de la moyenne spatiale France...")
    print("   ⚙️ Calcul en cours (cela peut prendre 2-5 minutes)...")

    ds_mean = ds_france.mean(dim=['latitude', 'longitude'], skipna=True)
    ds_mean = ds_mean.compute()

    df = ds_mean.to_dataframe()
    df = df.reset_index()

    # Correction des noms de colonnes
    if 'valid_time' in df.columns:
        df = df.rename(columns={'valid_time': 'time'})

    if 'time' not in df.columns:
        df['time'] = df.iloc[:, 0]

    print("👀 Colonnes détectées :", df.columns.tolist())

    # --- SAUVEGARDE CSV ---
    if 't2m' in df.columns:
        df['Temperature_C'] = df['t2m'] - 273.15
        final_df = df[['time', 'Temperature_C']].sort_values('time')
        final_df.to_csv("donnees_meteo_france_25ans.csv", index=False)
        print(f"✅ Fichier CSV créé : 'donnees_meteo_france_25ans.csv'")
        print(f"   → {len(final_df)} mesures sauvegardées")
    else:
        print("❌ Erreur : Colonne 't2m' introuvable dans le DataFrame.")

    # --- FICHIER 2 : LA CARTE (NetCDF réduit) ---
    print("🗺️ Extraction d'une carte exemple...")

    try:
        # Prendre le milieu de la période (mois 150)
        mid_time = ds_france.valid_time[150]
        ds_carte = ds_france.sel(valid_time=mid_time, method="nearest")
        ds_carte = ds_carte.compute()
        ds_carte.to_netcdf("carte_exemple_25ans.nc")
        print(f"✅ Fichier NetCDF créé : 'carte_exemple_25ans.nc'")
        print(f"   → Date : {str(mid_time.values)[:10]}")
    except Exception as e_map:
        print(f"⚠️ Erreur lors de la création de la carte : {e_map}")

    # Fermeture propre
    ds.close()

    print("🎉 Traitement terminé avec succès !")
    print(f"📁 Fichiers créés dans : {os.getcwd()}")

except Exception as e:
    print(f"❌ ERREUR BLOQUANTE : {e}")
    import traceback
    print(traceback.format_exc())

🚀 Démarrage du script de récupération...
🔍 Sélection des fichiers...
   → 912 fichiers trouvés au total
   → Sélection de 300 fichiers (25 premières années)
   → Premier : era5_t2m_fr_1950_01.nc
   → Dernier : era5_t2m_fr_1974_12.nc
⏳ Chargement des données (cela peut prendre 1-2 minutes)...


c:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\.venv\Lib\site-packages\dask\_task_spec.py:758: UserWarning: The specified chunks separate the stored chunks along dimension "valid_time" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  return self.func(*new_argspec, **kwargs)
c:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\.venv\Lib\site-packages\dask\_task_spec.py:758: UserWarning: The specified chunks separate the stored chunks along dimension "valid_time" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  return self.func(*new_argspec, **kwargs)
c:\Users\margo\OneDrive\Documents\M1 SSD\Data Visualisation\Projet-Data-Visu\Projet_Data_Visu_Margot_Hanaa\.venv\Lib\site-packages\dask\_task_spec.py:758: UserWarning: The specified chunks separate the stored chunks along dime

: 